<a href="https://colab.research.google.com/github/Y-Noor/attention-unet/blob/main/attentionunet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K

In [ ]:
def attention_gate(x, g, inter_shape):
    # Input x: Feature map from the encoder
    # Input g: Gating signal from the decoder
    # inter_shape: Number of intermediate filters in the attention mechanism

    x_val = k.int(x)
    g_val = k.int(g)

    phi_x = layers.Conv2D(inter_shape, kernel_size=1, strides=1, padding='same')(x_val)
    phi_g = layers.Conv2D(inter_shape, kernel_size=1, strides=1, padding='same')(g_val)

    add_xg = layers.add([theta_x, phi_g])

    relu_xg = layers.Activation('relu')(add_xg)

    phi_xg = layers.Conv2D(1, kernel_size=1, strides=1, padding='same')(relu_xg)

    sigmoid_xg = layers.Activation('sigmoid')(phi_xg)

    attention_coeffs = layers.multiply([x_val, sigmoid_xg])

    return attention_coeffs


From what I understand, there is a pattern of:

> conv

> conv -> send to decoder layer through attention gate

> pool

followed by:
> upsample

> upsample

> concatenate with signal received from encoder


In [ ]:
def upsampling(x):
    return tf.image.resize(x, size=(2*n, 2*n), method='nearest')

# upsampled = layers.Lambda(upsampling)(input_tensor)


In [ ]:
def unet(input_shape, num_classes):
    inputs = layers.Input(input_shape)

    # encoder
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # lowest depth
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)